# Appendix B

### B.1 Importing Libraries

In [1]:
from selenium import webdriver
import pandas as pd
import numpy as np
from datetime import *
import pickle 

### B.2 Specifying Chrome as Web Driver

In [2]:
#Experimenting with the packages 
driver  = webdriver.Chrome()

### B.3 Data Retrival Functions

In [24]:
#Main functions used for web scrapping



#Function list_maker 
#input column HTML column to convert into a python list
#outputs python list of strings for the specified HTML column 
def list_maker(column):
    make = []
    for i in range(len(column)):
        make.append(column[i].text)
    return make

#WebsSraper 
#inputs
#Date specific date to get data from
#driver specified selenium web driver to fetch data fram
#will use the driver element to refresh the current webpage and find the table within the page. 
#It will go through each column and fetch data for each row
#Then it will use lsit maker to make an appropriate list for every element stored in the table
#Outputs a dictionary with all the column headers and their values for that specific date
def WeatherScraper(Date, driver):
    driver.get('https://www.wunderground.com/history/daily/ca/london/CYXU/date/{0}'.format(Date))
    table = driver.find_element_by_xpath('//*[@id="inner-content"]/div[2]/div[1]/div[5]/div[1]/div/lib-city-history-observation/div/div[2]/table')
    headers = table.find_elements_by_xpath('//tr[2]/td')
    rData = [] 
    col = np.arange(1,20)
    for i in col:
        col = table.find_elements_by_xpath("//tr/td["+str(i)+"]")
        rData.append(col)
    Time = list_maker(rData[0])
    Temperature = list_maker(rData[1])
    DewPoint =list_maker(rData[2])
    Humidity = list_maker(rData[3])
    Wind=list_maker(rData[4])
    WindSpeed =list_maker(rData[5])
    WindGust=list_maker(rData[6])
    Pressure = list_maker(rData[7])
    Precip = list_maker(rData[8])
    Condition = list_maker(rData[9])

    temp_clean = Temperature[22:]
    time_clean = Time[22:]
    dew_clean =  DewPoint[22:]
    humidity_clean = Humidity[6:]
    Date = {'Temperature':temp_clean,'Time' :time_clean,'Dew':dew_clean, 'Humidity' : humidity_clean, 'Wind' :Wind
             ,'WindSpeed': WindSpeed,'WindGust' : WindGust,'Pressure' : Pressure,'Precip' : Precip, 'Condition' :Condition}
    return Date


#Function DateRangeCollector 
#inputs 
#start starting date
#end ending date 
#file file name to save to
#This is how we itterate through the webpage in full.
#It will use function WebScrapper to collect the data for the given end and start range
#it will then dump the returning dictionary into a pickel file to be available for retrieving later.
#Trouble shooting is done within the function so if for some reason the data is unavailable when webscrapper is trying to retrieve it
#this will dump the current dictionary into the file and print out a statement indicating what day the error occured in to 
#allow the proccess to be resumed in that day. 
#Outputs day_dict a dictionary of a ll the weather data

def DateRangeCollector(start,end,file):
    try:
        f = open(f"{file}","wb")
        delta =  end - start
        day_dict  = {}
        driver  = webdriver.Chrome()
        driver.implicitly_wait(3)
        day = start
        for i in range (delta.days + 1):
            day = start + timedelta(i)
            string_format = day.date().isoformat()
            test = WeatherScraper(string_format,driver)
            day_dict[string_format] =  test
        pickle.dump(day_dict,f)
        f.close()
    except:
        print('Error occured during this this date {0}'.format(string_format) )
        pickle.dump(day_dict,f)
        f.close()
        
        
    return day_dict
        
    

### B.4 Data Exportation

In [27]:
#Function calls to collect the days 
start =datetime(2020,1,21)
end =datetime(2020,4,1)
dictionary_of_days = DateRangeCollector(start,end,'Weather2020_jan21')
